In [1]:
import pandas as pd
import numpy as np
import glob
import functools

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

In [2]:
RAW_FILE = 'NHANES-varnames_raw.xlsx'
NO_FLAG_FILE = 'NHANES-varnames_noflag.csv'
YES_FLAG_FILE = 'NHANES-varnames_yesflag.csv'
CLEAN_FILE = 'NHANES-clean.csv'
MISSING_FILE = 'NHANES-missing.csv'

COMPONENTS = ['Laboratory', 'Demographics', 'Questionnaire', 'Dietary', 'Examination']

YEARS = list(range(2003, 2016, 2))

In [3]:
# read raw file and append sheets 
no_flag_df = pd.DataFrame()
for c in COMPONENTS: 
    df = pd.read_excel(RAW_FILE, sheet_name = c)
    no_flag_df = no_flag_df.append(df)
    
# create year flags 
def var_in_year(row, year): 
    if (row['Begin Year'] <= year) & (row['EndYear'] >= year): 
        return 1
    return 0 

for y in YEARS: 
    v = 'flag_' + str(y)
    no_flag_df[v] = no_flag_df.apply(var_in_year, year = y, axis = 1)
    
flags = [col for col in no_flag_df if col.startswith('flag_')]
no_flag_df = (no_flag_df
              .groupby(['Variable Name', 
                        'Variable Description', 
                        'Component', 
                        'Data File Description'])[flags]
              .sum(axis = 1)
              .reset_index())

for y in YEARS: 
    v = 'flag_' + str(y)
    no_flag_df[v] = np.where(no_flag_df[v] == 0, 0, 1)

# write to csv
no_flag_df.to_csv(NO_FLAG_FILE, index = False)

In [7]:
# get list of variables to keep 
yes_flag_df = pd.read_csv(YES_FLAG_FILE)
keep_vars = (yes_flag_df[yes_flag_df['keep'] == 1]['Variable Name']
             .unique()
             .tolist())
keep_vars = set(keep_vars)

# loop over all years and components 
full_df_list = []
for y in YEARS:
    print(y)
    year_df_list = []
    for c in COMPONENTS: 
        print(c)
        path = 'csv_data/' + str(y) + '-' + str(y+1) + '/' + c + '/*.csv'
        print(path)
        for f in glob.iglob(path):
            df = pd.read_csv(f)
            df = df[df.columns[df.columns.isin(keep_vars)]]
            if (df.shape[1] > 1) & ('SEQN' in df.columns): 
                year_df_list.append(df)
                
    #merge (wide) within each year
    year_df = (functools.reduce(
        lambda df1, df2: pd.merge(df1, df2, on = 'SEQN', how = 'outer'), year_df_list)
               .drop_duplicates(subset = 'SEQN'))
    year_df['year'] = str(y) + '-' + str(y+1)
    year_df = year_df.loc[:,~year_df.columns.duplicated()]
    full_df_list.append(year_df)

# append (long) across years 
clean_df = pd.concat(full_df_list, axis = 0, sort = True)

# combine DRSTZ variables 
clean_df['DR1DRSTZ'] = np.where(clean_df['DR1DRSTZ_x'].notna(), 
                                clean_df['DR1DRSTZ_x'], clean_df['DR1DRSTZ_y'])
clean_df['DR2DRSTZ'] = np.where(clean_df['DR2DRSTZ_x'].notna(), 
                                clean_df['DR2DRSTZ_x'], clean_df['DR2DRSTZ_y'])
clean_df = clean_df.drop(columns = ['DR1DRSTZ_x', 'DR1DRSTZ_y', 
                                    'DR2DRSTZ_x', 'DR2DRSTZ_y'])

# combine day 1 and day 2 variables - based on DR1 and DR2 prefixes 
exclude_cols = ['year', 'SEQN', 
                'DR1DRSTZ','DR2DRSTZ', 
                'DR1_300', 'DR2_300', 
                'RIDAGEYR', 'RIAGENDR', 'INDFMPIR', 'RIDRETH1', 'BMXBMI','BPQ020']

feature_cols = set()
for i in clean_df.columns: 
    if i not in exclude_cols: 
        feature_cols.add(i[3:])
    
for i in feature_cols: 
    clean_df[i] = clean_df['DR1' + i] + clean_df['DR2' + i]
    clean_df = clean_df.drop(columns = ['DR1' + i, 
                                        'DR2' + i])
    
# format and output to csv 
clean_df = round(clean_df, 5) 
clean_df.to_csv(CLEAN_FILE, index = False)

2003
Laboratory
csv_data/2003-2004/Laboratory/*.csv
Demographics
csv_data/2003-2004/Demographics/*.csv
Questionnaire
csv_data/2003-2004/Questionnaire/*.csv
Dietary
csv_data/2003-2004/Dietary/*.csv
Examination
csv_data/2003-2004/Examination/*.csv
2005
Laboratory
csv_data/2005-2006/Laboratory/*.csv
Demographics
csv_data/2005-2006/Demographics/*.csv
Questionnaire
csv_data/2005-2006/Questionnaire/*.csv
Dietary
csv_data/2005-2006/Dietary/*.csv
Examination
csv_data/2005-2006/Examination/*.csv
2007
Laboratory
csv_data/2007-2008/Laboratory/*.csv
Demographics
csv_data/2007-2008/Demographics/*.csv
Questionnaire
csv_data/2007-2008/Questionnaire/*.csv
Dietary
csv_data/2007-2008/Dietary/*.csv
Examination
csv_data/2007-2008/Examination/*.csv
2009
Laboratory
csv_data/2009-2010/Laboratory/*.csv
Demographics
csv_data/2009-2010/Demographics/*.csv
Questionnaire
csv_data/2009-2010/Questionnaire/*.csv
Dietary
csv_data/2009-2010/Dietary/*.csv
Examination
csv_data/2009-2010/Examination/*.csv
2011
Laboratory


In [6]:
clean_df

,BMXBMI,BPQ020,DR1TACAR,DR1TATOA,DR1TB12A,DR1TBCAR,DR1TCAFF,DR1TCARB,DR1TCHL,DR1TCHOL,DR1TCOPP,DR1TCRYP,DR1TFA,DR1TFDFE,DR1TFF,DR1TFIBE,DR1TIRON,DR1TLYCO,DR1TM161,DR1TM181,DR1TM201,DR1TM221,DR1TMAGN,DR1TMFAT,DR1TP182,DR1TP183,DR1TP184,DR1TP204,DR1TP205,DR1TP226,DR1TPFAT,DR1TPHOS,DR1TPOTA,DR1TPROT,DR1TRET,DR1TS040,DR1TS060,DR1TS080,DR1TS100,DR1TS120,DR1TS140,DR1TS160,DR1TS180,DR1TSFAT,DR1TSUGR,DR1TTFAT,DR1TTHEO,DR1TVARA,DR1TVB1,DR1TVB2,DR1TVB6,DR1TVK,DR1_300,DR2TACAR,DR2TATOA,DR2TB12A,DR2TBCAR,DR2TCAFF,DR2TCARB,DR2TCHL,DR2TCHOL,DR2TCOPP,DR2TCRYP,DR2TFA,DR2TFDFE,DR2TFF,DR2TFIBE,DR2TIRON,DR2TLYCO,DR2TM161,DR2TM181,DR2TM201,DR2TM221,DR2TMAGN,DR2TMFAT,DR2TP182,DR2TP183,DR2TP184,DR2TP204,DR2TP205,DR2TP226,DR2TPFAT,DR2TPHOS,DR2TPOTA,DR2TPROT,DR2TRET,DR2TS040,DR2TS060,DR2TS080,DR2TS100,DR2TS120,DR2TS140,DR2TS160,DR2TS180,DR2TSFAT,DR2TSUGR,DR2TTFAT,DR2TTHEO,DR2TVARA,DR2TVB1,DR2TVB2,DR2TVB6,DR2TVK,DR2_300,INDFMPIR,RIAGENDR,RIDAGEYR,RIDRETH1,SEQN,year,DR1DRSTZ,DR2DRSTZ,TZINC,TVC,TNIAC,TCALC,TALCO,TLZ,TSELE,TVB12,TVD,TFOLA,TKCAL,TP225,TMOIS,TATOC,TSODI
0,50.85,2.0,4.400000e+01,5.397605e-79,5.397605e-79,1.480000e+02,5.397605e-79,459.27,NaN,2.030000e+02,1.294,1.800000e+01,1.380000e+02,427.0,1.930000e+02,1.670000e+01,13.17,5.397605e-79,7.370000e-01,31.809,3.230000e-01,3.300000e-02,272.0,33.006,34.888,1.452,5.397605e-79,9.600000e-02,1.100000e-02,3.300000e-02,36.546,993.0,2432.0,67.17,1.110000e+02,2.720000e-01,1.720000e-01,1.020000e-01,2.140000e-01,0.316,0.936,16.293,5.837,24.718,258.71,101.42,5.397605e-79,127.0,2.416,1.498,1.733,58.2,2.0,6.200000e+01,5.397605e-79,5.397605e-79,4.980000e+02,5.397605e-79,2.397300e+02,NaN,3.000000e+02,0.931,9.000000e+00,1.290000e+02,3.280000e+02,1.080000e+02,6.900000e+00,1.306000e+01,5.303000e+03,2.243000e+00,3.168200e+01,2.500000e-01,1.870000e-01,164.0,3.497500e+01,1.553600e+01,2.057000e+00,5.397605e-79,1.790000e-01,5.397605e-79,1.900000e-02,1.894600e+01,1.292000e+03,2.052000e+03,1.046600e+02,1.550000e+02,5.140000e-01,2.650000e-01,1.590000e-01,3.320000e-01,3.720000e-01,2.443000e+00,1.752800e+01,8.825000e+00,3.154600e+01,1.400100e+02,9.324000e+01,5.397605e-79,2.000000e+02,1.789000e+00,1.611000e+00,1.763000e+00,3.380000e+01,2.0,2.440000e+00,1.0,1.900000e+01,4.0,21005.0,2003-2004,1.0,1.0,24.80,163.5,50.199,1195.0,1.079521e-78,1.321000e+03,213.6,7.38,NaN,567.0,5195.0,3.100000e-02,3683.42,14.34,10378.0
425,20.78,2.0,1.100000e+01,5.397605e-79,5.397605e-79,1.300000e+02,5.397605e-79,168.55,NaN,4.630000e+02,0.567,1.700000e+01,7.900000e+01,210.0,7.600000e+01,7.800000e+00,19.18,2.482000e+03,6.270000e-01,17.048,1.260000e-01,1.700000e-02,123.0,18.048,6.180,0.588,5.397605e-79,1.570000e-01,5.000000e-03,3.600000e-02,6.987,611.0,994.0,33.06,2.330000e+02,3.390000e-01,1.940000e-01,1.560000e-01,2.350000e-01,0.357,1.168,7.681,3.754,14.409,77.72,43.00,5.397605e-79,245.0,0.534,1.134,0.688,12.6,3.0,5.397605e-79,5.397605e-79,5.397605e-79,7.500000e+01,5.397605e-79,1.842300e+02,NaN,6.600000e+01,0.527,8.000000e+00,7.100000e+01,1.720000e+02,5.100000e+01,8.000000e+00,6.630000e+00,3.224000e+03,6.040000e-01,1.724100e+01,3.300000e-02,5.397605e-79,116.0,1.800400e+01,5.527000e+00,4.700000e-01,5.397605e-79,1.600000e-02,5.397605e-79,5.397605e-79,6.027000e+00,7.050000e+02,1.229000e+03,3.471000e+01,2.420000e+02,3.520000e-01,1.500000e-01,1.010000e-01,2.060000e-01,2.320000e-01,1.180000e+00,6.904000e+00,3.440000e+00,1.279400e+01,7.149000e+01,3.970000e+01,5.397605e-79,2.480000e+02,1.176000e+00,1.047000e+00,9.520000e-01,1.100000e+01,1.0,2.470000e+00,2.0,1.600000e+01,4.0,21006.0,2003-2004,1.0,1.0,8.96,144.3,20.584,898.0,1.079521e-78,5.490000e+02,105.1,2.92,NaN,277.0,2396.0,1.079521e-78,1917.79,5.99,4528.0
495,18.43,NaN,5.000000e+00,5.397605e-79,5.397605e-79,2.180000e+02,2.700000e+01,186.03,NaN,4.800000e+01,0.607,1.650000e+02,1.280000e+02,285.0,6.800000e+01,5.000000e+00,8.22,7.139000e+03,5.100000e-01,12.992,5.800000e-02,5.397605e-79,81.0,13.591,6.823,0.561,1.200000e-02,3.400000e-02,5.397605e-79,5.397605e-79,7.441,467.0,768.0,28.79,1.730000e+02,1.440000e-01,